In [ ]:
pip install beautifulsoup4 pandas html5lib

In [ ]:
pip install --upgrade html5lib lxml


In [39]:
import pandas as pd
from bs4 import BeautifulSoup
import re

class AuditReportParser:
    def parse_html(self, file_path):
        """
        HTML 파일을 읽고 BeautifulSoup 객체로 파싱하여 반환합니다.
        
        Args:
            file_path (str): HTML 파일 경로.
            
        Returns:
            BeautifulSoup: 파싱된 HTML 객체.
            
        Raises:
            IOError: 파일을 읽을 수 없을 때 발생.
        """
        try:
            # 감사보고서 파일의 인코딩이 'euc-kr'인 경우가 많습니다.
            with open(file_path, 'r', encoding='euc-kr') as f:
                html_content = f.read()
            return BeautifulSoup(html_content, 'html.parser')
        except IOError as e:
            print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
            return None
    
    def extract_sections(self, parsed_html):
        """
        HTML에서 <p> 태그의 텍스트를 문단별로 추출하여 리스트로 반환합니다.
        
        Args:
            parsed_html (BeautifulSoup): parse_html() 메서드가 반환한 객체.
            
        Returns:
            list: 각 <p> 태그의 텍스트가 담긴 문자열 리스트.
        """
        if not parsed_html:
            return []
            
        paragraphs = parsed_html.find_all('p')
        parsed_texts = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]
        
        return parsed_texts
    
    def extract_tables(self, parsed_html):
        """
        HTML에서 class="TABLE" 속성을 가진 테이블을 DataFrame 리스트로 추출합니다.
        
        Args:
            parsed_html (BeautifulSoup): parse_html() 메서드가 반환한 객체.
            
        Returns:
            list: 각 테이블을 변환한 pandas DataFrame 객체 리스트.
        """
        if not parsed_html:
            return []
            
        tables = parsed_html.find_all('table', class_='TABLE')
        dfs = []
        
        for table in tables:
            rows = table.find_all('tr')
            
            if not rows:
                continue
                
            data = []
            max_cols = 0
            for row in rows:
                cols = row.find_all(['td', 'th'])
                row_data = [col.get_text(strip=True) for col in cols]
                data.append(row_data)
                if len(row_data) > max_cols:
                    max_cols = len(row_data)
            
            padded_data = [row + [''] * (max_cols - len(row)) for row in data]
            
            if padded_data and len(padded_data) > 1:
                df = pd.DataFrame(padded_data[1:], columns=padded_data[0])
                dfs.append(df)
            elif padded_data:
                df = pd.DataFrame(padded_data)
                dfs.append(df)
        
        return dfs

    def normalize_text(self, text):
        """
        금융 텍스트를 정규화하고 정제합니다.
        
        Args:
            text (str): 정제할 텍스트.
            
        Returns:
            str: 정규화된 텍스트.
        """
        # 불필요한 공백 문자(줄바꿈, 탭 등)를 단일 공백으로 치환
        text = re.sub(r'\s+', ' ', text)
        # 괄호와 그 안의 내용을 제거 (예: (주)삼성전자)
        text = re.sub(r'\([^)]*\)', '', text)
        # 특수문자 제거 (한글, 영어, 숫자, 공백 제외)
        text = re.sub(r'[^가-힣a-zA-Z0-9\s.,]', '', text)
        # 쉼표 뒤에 공백 추가
        text = re.sub(r',', ', ', text)
        # 여러 개의 공백을 단일 공백으로
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text

    def extract_inventory_table(self, parsed_html,year): #재고자산 내역 테이블
        """
        HTML에서 '재고자산 내역' 테이블을 찾아 DataFrame으로 반환합니다.
        """
        search_texts = [
            '보고기간종료일 현재 재고자산의 내역은 다음과 같습니다.',
            '재고자산의 내역은 다음과 같습니다.',
        ]

        for text in search_texts:
            title_element = parsed_html.find(
                lambda tag: tag.name in ['p', 'span'] and text in tag.get_text(strip=True)
            )
            
            if title_element:
                table_element = title_element.find_next('table', class_='TABLE')
                
                if table_element:
                    rows = table_element.find_all('tr')
                    
                    # 헤더 추출 로직
                    # 첫 번째 행과 두 번째 행의 헤더를 결합하여 최종 컬럼 이름 생성
                    header_row1 = rows[0].find_all(['th', 'td'])
                    header_row2 = rows[1].find_all(['th', 'td'])
                    
                    headers = [header_row1[0].get_text(strip=True)]
                    
                    # '당기말' 헤더와 그 아래 세부 헤더 결합
                    headers.extend([
                        f'{header_row1[1].get_text(strip=True)} {h.get_text(strip=True)}' 
                        for h in header_row2[0:3]
                    ])

                    # '전기말'에 해당하는 헤더 3개는 제외
                    # `header_row2[3:6]`에 해당하는 열은 무시함.
                    
                    # 데이터 추출 로직
                    data_rows = rows[2:]
                    data = []
                    for row in data_rows:
                        cols = row.find_all('td')
                        # 첫 번째 열('구분')과 다음 세 개 열('당기말' 관련)만 선택
                        row_data = [cols[0].get_text(strip=True)] + [td.get_text(strip=True) for td in cols[1:4]]
                        data.append(row_data)

                    df = pd.DataFrame(data, columns=headers)
                    
                    df.columns = df.columns.str.replace(r'\s+', '', regex=True)
                    # '장부금액'을 '장부가액'으로 통일
                    df.columns = df.columns.str.replace('장부금액', '장부가액')

                    df.columns = df.columns.str.replace(r'[\(\)\*]', '', regex=True).str.strip()

                    for col in df.columns:
                        df[col] = df[col].astype(str).str.replace(r'[\(\)\*]', '', regex=True).str.strip()

                    # '연도' 컬럼 추가
                    df['연도'] = year
                    df['단위'] = '백만원'
                    cols = ['연도'] + [col for col in df.columns if col != '연도']
                    df = df[cols]
                    
                    return df
                    
        return None


    def extract_investment_changes(self, parsed_html,year): #종속기업, 관계기업 및 공동기업 투자의 변동내역 테이블
        """
        HTML에서 '종속기업, 관계기업 및 공동기업 투자의 변동내역' 테이블을 찾아 DataFrame으로 반환합니다.
        
        Args:
            parsed_html (BeautifulSoup): 파싱된 HTML 객체.
            
        Returns:
            pd.DataFrame: 변동내역 데이터가 담긴 DataFrame.
        """
        # 텍스트를 포함하는 p 또는 span 태그를 찾습니다.
        search_text = '가. 당기 및 전기 중 종속기업, 관계기업 및 공동기업 투자의 변동'
        
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_text in tag.get_text(strip=True)
        )

        if not title_element:
            print(f"'{search_text}' 텍스트를 찾을 수 없습니다.")
            return None
        
        # 텍스트 다음에 나오는 첫 번째 'TABLE' 클래스 테이블을 찾습니다.
        table_element = title_element.find_next('table', class_='TABLE')

        if not table_element:
            print("관련 테이블을 찾을 수 없습니다.")
            return None
        
        # 테이블 데이터 추출
        rows = table_element.find_all('tr')
        
        if len(rows) < 2:
            print("테이블에 충분한 데이터가 없습니다.")
            return None
            
        # 헤더와 데이터 추출
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]

        # DataFrame 생성 및 반환
        df = pd.DataFrame(data, columns=headers)
        df = df.rename(columns={'당기': '변동내역'})
        if '전기' in df.columns:
            df = df.drop(columns=['전기'])
            
        df.columns = df.columns.str.replace(r'[\s\(\)\*]', '', regex=True)

        for col in df.columns:
            # '구분' 열은 숫자까지 제거
            if col == '구분':
                df[col] = df[col].astype(str).str.replace(r'[\d\(\)\*\s]', '', regex=True).str.strip()
            # 그 외 열은 괄호, 별표, 공백만 제거
            else:
                df[col] = df[col].astype(str).str.replace(r'[\(\)\*\s]', '', regex=True).str.strip()

        df['연도'] = year
        df['단위'] = '백만원'
        cols = ['연도'] + [col for col in df.columns if col != '연도']
        df = df[cols]
        return df
    
    def extract_major_investments(self, parsed_html,year): #관계기업 투자 현황 테이블
        """
        HTML에서 '주요 관계기업 및 공동기업 투자 현황' 테이블을 찾아 DataFrame으로 반환합니다.
        
        Args:
            parsed_html (BeautifulSoup): 파싱된 HTML 객체.
            
        Returns:
            pd.DataFrame: 주요 투자 현황 데이터가 담긴 DataFrame.
        """
        search_texts = [
            '(1) 관계기업 투자',
            '투자 현황은 다음과 같습니다'
        ]
        
        for text in search_texts:
            title_element = parsed_html.find(
                lambda tag: tag.name in ['p', 'span'] and text in tag.get_text(strip=True)
            )
            
            if title_element:
                table_element = title_element.find_next('table', class_='TABLE')
                if table_element:
                    rows = table_element.find_all('tr')
                    if len(rows) < 2:
                        continue
                        
                    headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
                    data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
                    
                    df = pd.DataFrame(data, columns=headers)
                    df.columns = df.columns.str.replace(r'\s+', '', regex=True)

                    df.columns = df.columns.str.replace(r'[\d\(\)\*]', '', regex=True).str.strip()
                    
                    for col in df.columns:
                        df[col] = df[col].astype(str).str.replace(r'[\(\)\*]', '', regex=True).str.strip()
                    if '결산월' in df.columns:
                        df = df.drop(columns=['결산월'])
                    # 지분율 컬럼 전처리
                    if '지분율%' in df.columns:
                        df['지분율%'] = pd.to_numeric(df['지분율%'], errors='coerce')
                        
                    for col in df.columns:
                        df[col] = df[col].astype(str).str.replace(r'[\(\)]', '', regex=True)
                    df = df.rename(columns={'관계기업과의관계의성격': '관계의성격'})
                    df['연도'] = year
                    cols = ['연도'] + [col for col in df.columns if col != '연도']
                    df = df[cols]
                    return df
        
        print("주요 관계기업 투자 현황 테이블을 찾을 수 없습니다.")
        return None

    def extract_subsidiaries(self, parsed_html,year): #종속기업 재무정보 테이블
        """
        HTML에서 '(1) 주요 종속기업' 테이블을 찾아 두 가지 유형의 헤더를 모두 처리하여 DataFrame으로 반환합니다.
        """
        search_text = '(1) 주요 종속기업'
        
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_text in tag.get_text(strip=True)
        )

        if not title_element:
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        
        if not table_element:
            return None
        
        rows = table_element.find_all('tr')
        
        if len(rows) < 2:
            return None
            
        # 첫 번째 행의 'rowspan' 속성으로 다중 헤더 여부 판단
        first_row_ths = rows[0].find_all(['th', 'td'])
        is_multi_header = any(th.get('rowspan') for th in first_row_ths)

        # 헤더와 데이터 추출
        if is_multi_header and len(rows) >= 2:
            # 다중 헤더 처리: 두 번째 행의 헤더를 사용
            # 첫 번째 행은 버리고 두 번째 행을 헤더로 사용
            lower_header_elements = rows[1].find_all(['th', 'td'])
            headers = [th.get_text(strip=True) for th in lower_header_elements]
            
            # 첫 번째 열에 해당하는 헤더가 없으면 '기업명'으로 채움
            if len(headers) != len(rows[2].find_all('td')):
                headers = ['기업명'] + [h for h in headers]
                
            data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[2:]]
            
            df = pd.DataFrame(data, columns=headers)
            
        else:
            # 단일 헤더 처리
            headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
            data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
            
            df = pd.DataFrame(data, columns=headers)
            
        # 공통 전처리: 불필요한 텍스트 및 기호 제거
        df.columns = df.columns.astype(str).str.replace(r'\(.*?\)', '', regex=True).str.strip()
        df.rename(columns={'기업명': '기업명', '당기순이익(손실)': '당기순이익'}, inplace=True)
        
        # 숫자형 컬럼 전처리 (콤마와 하이픈 제거 후 float 변환)
        for col in df.columns[1:]:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '').str.replace('-', '0'), errors='coerce')
        for col in df.columns:
            df[col] = df[col].astype(str).str.replace(r'[\(\)]', '', regex=True)
        df['연도'] = year
        df['단위'] = '백만원'
        cols = ['연도'] + [col for col in df.columns if col != '연도']
        df = df[cols]
        return df
    
    def _parse_table_content(self, table_element):# 관계기업 재무정보 테이블
        """BeautifulSoup 테이블 객체를 DataFrame으로 파싱하는 도우미 함수."""
        if not table_element:
            return None
        rows = table_element.find_all('tr')
        if not rows or len(rows) < 2:
            return None
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        return pd.DataFrame(data, columns=headers)

        
    def extract_financial_info_double(self, parsed_html):# 관계기업 재무정보 테이블
        """관계기업 재무정보 테이블을 파싱하여 DataFrame으로 반환합니다."""
        search_pattern_main = re.compile(r'\(2\)\s*주요\s*관계기업')
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and re.search(search_pattern_main, tag.get_text(strip=True))
        )
        if not title_element:
            print("경고: '주요 관계기업' 제목을 찾을 수 없습니다.")
            return None
            
        # 두 개의 테이블이 연속해서 있는지 확인하는 로직
        tables_after_title = title_element.find_all_next('table', class_='TABLE', limit=2)
        is_two_tables_case = len(tables_after_title) >= 2

        if is_two_tables_case:
            table_element_1 = tables_after_title[0]
            table_element_2 = tables_after_title[1]

            if not table_element_1 or not table_element_2:
                print("경고: 두 개의 분리된 테이블을 찾을 수 없습니다.")
                return None
            
            df_bs = self._parse_table_content(table_element_1)
            df_is = self._parse_table_content(table_element_2)

            if df_bs is None or df_is is None:
                return None
            
            samsung_card_col_name = next((col for col in df_bs.columns if '삼성카드' in col), None)
            if samsung_card_col_name and '유동자산' in df_bs['구분'].values and '비유동자산' in df_bs['구분'].values:
                print("삼성카드 데이터 보정 중...")
                df_bs.loc[df_bs['구분'] == '비유동자산', samsung_card_col_name] = df_bs.loc[df_bs['구분'] == '유동자산', samsung_card_col_name].values[0]
                df_bs.loc[df_bs['구분'] == '비유동부채', samsung_card_col_name] = df_bs.loc[df_bs['구분'] == '유동부채', samsung_card_col_name].values[0]
            
            df_bs.set_index('구분', inplace=True)
            df_is.set_index('구분', inplace=True)
            final_df = pd.concat([df_bs, df_is], axis=0, join='outer')

                    
           # for col in final_df.columns:
                #final_df[col] = final_df[col].astype(str).str.replace(r'[\(\)\*]', '', regex=True).str.strip()

            final_df.index.name = None
            final_df = final_df.T
            
            final_df.columns = final_df.columns.str.replace(r'[\(\)\*]', '', regex=True)

            final_df.index = final_df.index.str.replace(r'[\s\(\)\*]', '', regex=True)

            for col in final_df.columns:
                final_df[col] = final_df[col].astype(str).str.replace(r'[\(\)]', '', regex=True)
            final_df.columns = final_df.columns.str.replace('수익매출', '매출')
            return final_df

        
    def extract_financial_info_single(self, parsed_html):# 관계기업 재무정보 테이블
        """
        통합된 '관계기업의 재무정보' 테이블을 파싱하여 DataFrame으로 반환합니다.
        """
        search_pattern = re.compile(r'\(2\)\s*주요\s*관계기업')
        
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and re.search(search_pattern, tag.get_text(strip=True))
        )
        if not title_element:
            print("경고: '주요 관계기업' 제목을 찾을 수 없습니다.")
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        if not table_element:
            print("경고: 관련 테이블을 찾을 수 없습니다.")
            return None
            
        rows = table_element.find_all('tr') 
        if len(rows) < 2:
            print("경고: 테이블에 충분한 행이 없습니다.")
            return None

        # 헤더 행의 개수를 기준으로 파싱 로직 분기
        if len(rows) >= 3 and len(rows[1].find_all(['th', 'td'])) > 1:
            # 헤더 행이 2개인 경우
            company_names_row = rows[1]
            data_start_row = 2
        else:
            # 헤더 행이 1개인 경우
            company_names_row = rows[0]
            data_start_row = 1
        
        # 1. 기업명(컬럼) 추출: <th>와 <td> 태그를 모두 찾아 '구분'을 제외
        company_names = []
        for elem in company_names_row.find_all(['th', 'td']):
            elem_text = elem.get_text(strip=True)
            if '구분' not in elem_text:
                company_names.append(elem_text)
        
        # 2. 데이터 및 재무 항목(인덱스) 추출
        financial_items = []
        financial_data = []

        clean_pattern = re.compile(r'[\s\(\)\*]')

        for row in rows[data_start_row:]:
            tds = row.find_all('td')
            if '요약' in tds[0].get_text(strip=True):
                continue
            
            item_name = re.sub(clean_pattern, '', tds[0].get_text(strip=True))
            item_data = [td.get_text(strip=True) for td in tds[1:]]
            
            if len(item_data) != len(company_names):
                print(f"경고: 데이터 행의 열 개수({len(item_data)})와 기업명 개수({len(company_names)})가 불일치합니다.")
                continue

            financial_items.append(item_name)
            financial_data.append(item_data)
        
        if not financial_data:
            print("경고: 유효한 재무 데이터 행을 찾을 수 없습니다. DataFrame 생성 실패.")
            return None
        
        # 3. DataFrame 생성
        df = pd.DataFrame(financial_data, columns=company_names, index=financial_items)
        df.columns = df.columns.str.replace(r'\s+', '', regex=True)

        #df.columns = df.columns.str.replace(r'[\d\(\)\*]', '', regex=True).str.strip()
                    
        for col in df.columns:
            df[col] = df[col].astype(str).str.replace(r'[\(\)\*]', '', regex=True).str.strip()
        # 4. '순자산' 열 추가 (요청에 따라)
        
        # 5. 숫자형 데이터 전처리
        for col in df.columns:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '').str.replace('-', '0').str.replace('(', '-').str.replace(')', ''), errors='coerce')
        
        # 최종적으로 행과 열을 뒤집어 원하는 형태로 만듭니다.
        df = df.T
        for col in df.columns:
            df[col] = df[col].astype(str).str.replace(r'[\(\)]', '', regex=True)
        df.columns = df.columns.str.replace('수익매출', '매출')

        return df
    
    def extract_financial_info(self, parsed_html, year): # 관계기업 재무정보 테이블
        """
        관계기업 재무정보 테이블의 구조를 파악하고 적절한 파싱 메서드를 호출합니다.
        """
        if year >= 2019:
            df =self.extract_financial_info_single(parsed_html)
            df['연도'] = year
            df['단위'] = '백만원'
            cols = ['연도'] + [col for col in df.columns if col != '연도']
            df = df[cols]
            return df
        else:
            df =self.extract_financial_info_double(parsed_html)
            df['연도'] = year
            df['단위'] = '백만원'
            cols = ['연도'] + [col for col in df.columns if col != '연도']
            df = df[cols]
            return df
        
    def extract_specific_investment_table(self,parsed_html,year): #관계기업 투자주식의 내역 테이블
        """
        첫 번째 헤더 행을 무시하고, 두 번째 행을 헤더로 사용하여
        복합 헤더 테이블을 DataFrame으로 파싱합니다.
        
        Args:
            parsed_html (BeautifulSoup): 파싱된 HTML 객체.
            
        Returns:
            pd.DataFrame: 파싱된 데이터가 담긴 DataFrame.
        """
        search_text = "관계기업 투자주식의 내역은 다음과 같습니다."
    
        # 텍스트를 포함하는 태그를 찾습니다.
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_text in tag.get_text(strip=True)
        )
        
        if not title_element:
            print(f"'{search_text}' 텍스트를 찾을 수 없습니다.")
            return None
        
        table = title_element.find_next('table', class_='TABLE')
        
        if not table:
            print("class='TABLE'인 테이블을 찾을 수 없습니다.")
            return None

        # 모든 행을 추출합니다.
        rows = table.find_all('tr')
        
        if len(rows) < 2:
            print("테이블에 충분한 데이터가 없습니다.")
            return None
        
        # 두 번째 행을 헤더로 사용합니다.
        # 첫 번째 행은 무시합니다.
        headers = [th.get_text(strip=True) for th in rows[1].find_all(['th', 'td'])]
        
        # 세 번째 행부터 데이터로 사용합니다.
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[2:]]
        
        # '구분'에 해당하는 첫 번째 헤더가 누락되었을 수 있으므로 추가합니다.
        # 제공된 HTML 구조에서는 두 번째 행에 '구분' 헤더가 없으므로 수동으로 추가합니다.
        headers = ['구분'] + headers
        
        # 데이터 행도 첫 번째 열을 추가해야 함 (만약 테이블 구조에 따라 필요하다면)
        # 하지만 제공된 HTML에서는 데이터가 6개 열이므로 그대로 사용합니다.
        
        # 데이터프레임 생성
        df = pd.DataFrame(data, columns=headers)
        df.columns = df.columns.str.replace(r'\s+', '', regex=True)
        df.columns = df.columns.str.replace('장부가액', '장부금액', regex=True)
    
        df = df.rename(columns={
            '보유주식수(주)': '주식수(주)',
            '주식수': '주식수(주)'
        })

        df = df.iloc[:, :-2]
        for col in df.columns:
            df[col] = df[col].astype(str).str.replace(r'\*', '', regex=True)
        df['연도'] = year
        df['단위'] = '백만원'
        cols = ['연도'] + [col for col in df.columns if col != '연도']
        df = df[cols]
        return df
    
    
    def extract_tangible_assets_table(self,parsed_html,year): #유형자산 테이블
        # 띄어쓰기를 허용하는 정규표현식 패턴
        search_pattern = re.compile(r'유형자산의\s*변동\s*내역은\s*다음과\s*같습니다\.')

        # 정규표현식 패턴을 사용하여 텍스트가 포함된 태그를 찾습니다.
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_pattern.search(tag.get_text(strip=True))
        )
        
        if not title_element:
            print(f"'{search_pattern}' 텍스트를 찾을 수 없습니다.")
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        
        if not table_element:
            print("class='TABLE'인 테이블을 찾을 수 없습니다.")
            return None

        # 모든 행을 추출합니다.
        rows = table_element.find_all('tr')
        
        if len(rows) < 2:
            print("테이블에 충분한 데이터가 없습니다.")
            return None
        
        # 두 번째 행을 헤더로 사용합니다.
        # 첫 번째 행은 무시합니다.
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        
        # 세 번째 행부터 데이터로 사용합니다.
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        
        # '구분'에 해당하는 첫 번째 헤더가 누락되었을 수 있으므로 추가합니다.
        # 제공된 HTML 구조에서는 두 번째 행에 '구분' 헤더가 없으므로 수동으로 추가합니다.
 #       headers = ['구분'] + headers
        
        # 데이터 행도 첫 번째 열을 추가해야 함 (만약 테이블 구조에 따라 필요하다면)
        # 하지만 제공된 HTML에서는 데이터가 6개 열이므로 그대로 사용합니다.
        
        # 데이터프레임 생성
        df = pd.DataFrame(data, columns=headers)
        
         # DataFrame 헤더 이름을 먼저 표준화하여 '구분' 열을 보정
        df.columns = df.columns.str.replace(r'[\s\(\)]', '', regex=True)
        # '구분' 열이 '구 분'과 같이 공백이 있는 경우를 대비해 직접 변경
        if '구분' not in df.columns:
            df.rename(columns={col: '구분' for col in df.columns if '구분' in col.replace(' ', '')}, inplace=True)
            
        # 상위 항목을 저장할 변수 초기화
        parent_item = ""
        new_gu_bun = []

        # 데이터프레임의 행을 순회하며 '구분' 열 수정
        for index, row in df.iterrows():
            # '구분' 열이 존재한다는 가정 하에 작업
            current_item = row['구분']
            
            # '취득원가'나 '감가상각누계액'과 같은 하위 항목을 식별
            if '취득원가' in current_item or '감가상각누계액' in current_item:
                # 상위 항목이 있다면 결합하고, 아니면 현재 항목을 그대로 추가
                if parent_item:
                    new_gu_bun.append(f"{parent_item}/{current_item}")
                else:
                    new_gu_bun.append(current_item)
            
            else:
                # 하위 항목이 아닌 경우, 이 항목을 상위 항목으로 저장
                parent_item = current_item
                new_gu_bun.append(current_item)

        # 수정된 '구분' 열을 DataFrame에 적용
        df['구분'] = new_gu_bun

        df.columns = df.columns.str.replace(r'\s+', '', regex=True)
        for col in df.columns:
            df[col] = df[col].astype(str).str.replace(r'[\s\-\(\)\*]', '', regex=True).str.strip()
        df.columns = df.columns.str.replace(r'.*합계.*', '계', regex=True)

        df['연도'] = year
        df['단위'] = '백만원'
        cols = ['연도'] + [col for col in df.columns if col != '연도']
        df = df[cols]
        return df
    def extract_intangible_assets_table(self,parsed_html,year): #무형자산 테이블
        # 띄어쓰기를 허용하는 정규표현식 패턴
        search_pattern = re.compile(r'무형자산의\s*변동\s*내역은\s*다음과\s*같습니다\.')

        # 정규표현식 패턴을 사용하여 텍스트가 포함된 태그를 찾습니다.
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_pattern.search(tag.get_text(strip=True))
        )
        
        if not title_element:
            print(f"'{search_pattern}' 텍스트를 찾을 수 없습니다.")
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        
        if not table_element:
            print("class='TABLE'인 테이블을 찾을 수 없습니다.")
            return None

        # 모든 행을 추출합니다.
        rows = table_element.find_all('tr')
        
        if len(rows) < 2:
            print("테이블에 충분한 데이터가 없습니다.")
            return None
        
        # 두 번째 행을 헤더로 사용합니다.
        # 첫 번째 행은 무시합니다.
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        
        # 세 번째 행부터 데이터로 사용합니다.
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        
        # '구분'에 해당하는 첫 번째 헤더가 누락되었을 수 있으므로 추가합니다.
        # 제공된 HTML 구조에서는 두 번째 행에 '구분' 헤더가 없으므로 수동으로 추가합니다.
 #       headers = ['구분'] + headers
        
        # 데이터 행도 첫 번째 열을 추가해야 함 (만약 테이블 구조에 따라 필요하다면)
        # 하지만 제공된 HTML에서는 데이터가 6개 열이므로 그대로 사용합니다.
        
        # 데이터프레임 생성
        df = pd.DataFrame(data, columns=headers)
        df.columns = df.columns.str.replace(r'\s+', '', regex=True)
        for col in df.columns:
            df[col] = df[col].astype(str).str.replace(r'[\s\-\(\)\*]', '', regex=True).str.strip()
        df.columns = df.columns.str.replace(r'.*합계.*', '계', regex=True)

        df['연도'] = year
        df['단위'] = '백만원'
        cols = ['연도'] + [col for col in df.columns if col != '연도']
        df = df[cols]

        for col in df.columns:
            df[col] = df[col].astype(str).str.replace(r'[\(\)]', '', regex=True)
        return df

In [ ]:

# 클래스 인스턴스 생성
parser = AuditReportParser()

# 1. HTML 파일 파싱
file_path = '삼성전자_감사보고서_2014_2024/감사보고서_2014.htm'
parsed_html = parser.parse_html(file_path)

if parsed_html:
    # 2. 섹션별 텍스트 추출
    sections = parser.extract_sections(parsed_html)
    print("--- 추출된 문단 정보 (일부) ---")
    for i, section in enumerate(sections[:5]): # 상위 5개 문단만 출력
        normalized_section = parser.normalize_text(section)
        print(f"[{i+1}] {normalized_section}")
    print("-" * 20)
    
    # 3. 테이블 데이터 추출
    tables = parser.extract_tables(parsed_html)
    print("--- 추출된 테이블 정보 (첫 번째 테이블) ---")
    if tables:
        print(tables[0].head())
    else:
        print("파싱된 테이블이 없습니다.")

In [ ]:
if parsed_html:
    # 2. 섹션별 텍스트 추출 및 정규화
    sections = parser.extract_sections(parsed_html)
    print(f"총 {len(sections)}개의 문단이 파싱되었습니다.")
    
    print("\n--- 파싱된 문단 (일부) ---")
    for i, section in enumerate(sections[:10]):
        normalized_section = parser.normalize_text(section)
        print(f"[{i+1}] {normalized_section}")

In [ ]:
# 파일 경로 (사용자 로컬 환경에 맞춰 경로를 조정해야 합니다)
file_path = '삼성전자_감사보고서_2014_2024/감사보고서_2014.htm'

if os.path.exists(file_path):
    # 함수를 호출하여 모든 테이블을 추출합니다.
    all_tables_2014 = browse_tables(file_path)

    print(f"파일: {file_path}")
    print(f"총 {len(all_tables_2014)}개의 테이블이 파싱되었습니다.")
    
    # 추출된 테이블들을 함수 외부에서 순차적으로 출력합니다.
    for i, df in enumerate(all_tables_2014):
        print(f"\n--- [테이블 {i+1}] ---")
        display(df)
else:
    print(f"파일을 찾을 수 없습니다: {file_path}")

In [ ]:
import os
import re
import pandas as pd
from IPython.display import display

# AuditReportParser 클래스가 정의되어 있다고 가정합니다.
parser = AuditReportParser()

# 모든 DataFrame을 저장할 빈 리스트 생성
all_inventory_dfs = []

# 2014년부터 2024년까지 반복
for year in range(2014, 2025):
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
        year_from_filename = int(re.search(r'감사보고서_(\d{4})\.htm', file_path).group(1))
        
        if parsed_html:
            inventory_df = parser.extract_inventory_table(parsed_html, year_from_filename)
            
            print(f"\n==================== {year_from_filename}년 재고자산 ====================")
            if inventory_df is not None:
                display(inventory_df)
                
                # 추출된 DataFrame을 리스트에 추가
                all_inventory_dfs.append(inventory_df)
                
            else:
                print(f"재고자산 테이블을 찾을 수 없습니다.")
    else:
        print(f"\n파일을 찾을 수 없습니다: {file_path}")

# 루프가 끝난 후 모든 DataFrame을 하나로 합치기
if all_inventory_dfs:
    # ignore_index=True를 설정하여 합쳐진 DataFrame의 인덱스를 재설정합니다.
    final_inventory_df = pd.concat(all_inventory_dfs, ignore_index=True)
    
    # 최종 DataFrame을 CSV 파일로 저장
    final_csv_filename = "outputs/전체_재고자산_내역.csv"
    final_inventory_df.to_csv(final_csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"\n==================== 전체 재고자산 내역 ====================")
    display(final_inventory_df)
    print(f"'{final_csv_filename}' 파일로 저장되었습니다.")
else:
    print("\n추출된 데이터가 없어 파일을 저장할 수 없습니다.")

In [ ]:
import os
import re
import pandas as pd
from IPython.display import display

# parser 인스턴스는 이미 생성되었다고 가정합니다.
# parser = AuditReportParser()

# 출력 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# 모든 연도의 DataFrame을 저장할 빈 리스트
all_dfs = []

for year in range(2014, 2025):
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
        year_from_filename = int(re.search(r'감사보고서_(\d{4})\.htm', file_path).group(1))
        
        if parsed_html:
            df = parser.extract_investment_changes(parsed_html, year_from_filename)
            
            print(f"==================== {year_from_filename}년 투자의 변동내역 파싱 결과 ====================")
            if df is not None:
                display(df) 
                
                # 추출된 DataFrame을 리스트에 추가
                all_dfs.append(df)
            else:
                print("테이블을 찾을 수 없습니다.")
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")

# 루프가 끝난 후 모든 DataFrame을 하나로 합치기
if all_dfs:
    # pd.concat()을 사용해 리스트의 모든 DataFrame을 병합
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    # 최종 DataFrame을 CSV 파일로 저장
    final_csv_filename = "outputs/전체_종속기업,관계기업및공동기업투자_변동내역.csv"
    final_df.to_csv(final_csv_filename, index=False, encoding='utf-8-sig')
    
    print("\n==================== 전체 데이터 합치기 완료 ====================")
    display(final_df)
    print(f"'{final_csv_filename}' 파일로 저장되었습니다.")
else:
    print("\n추출된 데이터가 없어 파일을 저장할 수 없습니다.")

In [ ]:
import os
import re
import pandas as pd
from IPython.display import display

# parser 인스턴스는 이미 생성되었다고 가정합니다.
parser = AuditReportParser()

# 출력 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# 모든 연도의 DataFrame을 저장할 빈 리스트
all_dfs = []

for year in range(2014, 2025):
    # 파일 경로 설정
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    # 파일 존재 여부 확인 후 파싱 및 처리
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
        year_from_filename = int(re.search(r'감사보고서_(\d{4})\.htm', file_path).group(1))
        
        if parsed_html:
            df = parser.extract_major_investments(parsed_html, year_from_filename)
            
            print(f"==================== {year_from_filename}년 관계기업 투자 현황 파싱 결과 ====================")
            if df is not None:
                display(df) 
                
                # 추출된 DataFrame을 리스트에 추가
                all_dfs.append(df)
            else:
                print("테이블을 찾을 수 없습니다.")
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")

# 루프가 끝난 후 모든 DataFrame을 하나로 합치기
if all_dfs:
    # pd.concat()을 사용해 리스트의 모든 DataFrame을 병합합니다.
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    # 최종 DataFrame을 CSV 파일로 저장
    final_csv_filename = "outputs/전체_관계기업_투자_현황.csv"
    final_df.to_csv(final_csv_filename, index=False, encoding='utf-8-sig')
    
    print("\n==================== 전체 데이터 합치기 완료 ====================")
    display(final_df)
    print(f"'{final_csv_filename}' 파일로 저장되었습니다.")
else:
    print("\n추출된 데이터가 없어 파일을 저장할 수 없습니다.")

In [ ]:
import os
import re
import pandas as pd
from IPython.display import display

# parser 인스턴스는 이미 생성되었다고 가정합니다.
parser = AuditReportParser()

# 출력 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# 모든 연도의 DataFrame을 저장할 빈 리스트
all_dfs = []

for year in range(2014, 2025):
    # 파일 경로 설정
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
        year_from_filename = int(re.search(r'감사보고서_(\d{4})\.htm', file_path).group(1))
        
        if parsed_html:
            df = parser.extract_subsidiaries(parsed_html, year_from_filename)
            
            print(f"==================== {year_from_filename}년 종속기업 재무정보 파싱 결과 ====================")
            if df is not None:
                display(df) 
                
                # 추출된 DataFrame을 리스트에 추가
                all_dfs.append(df)
            else:
                print("테이블을 찾을 수 없습니다.")
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")

# 루프가 끝난 후 모든 DataFrame을 하나로 합치기
if all_dfs:
    # pd.concat()을 사용해 리스트의 모든 DataFrame을 병합합니다.
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    # 최종 DataFrame을 CSV 파일로 저장
    final_csv_filename = "outputs/전체_종속기업_재무정보.csv"
    final_df.to_csv(final_csv_filename, index=False, encoding='utf-8-sig')
    
    print("\n==================== 전체 데이터 합치기 완료 ====================")
    display(final_df)
    print(f"'{final_csv_filename}' 파일로 저장되었습니다.")
else:
    print("\n추출된 데이터가 없어 파일을 저장할 수 없습니다.")

In [ ]:
import os
import re
import pandas as pd
from IPython.display import display

# parser 인스턴스는 이미 생성되었다고 가정합니다.
# parser = AuditReportParser()

# 출력 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# 모든 연도의 DataFrame을 저장할 빈 리스트
all_dfs = []

for year in range(2014, 2025):
    # 파일 경로 설정
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    # 파일명에서 연도 추출
    match = re.search(r'(\d{4})', os.path.basename(file_path))
    current_year = int(match.group(1)) if match else None

    if os.path.exists(file_path) and current_year:
        parsed_html = parser.parse_html(file_path)
        
        if parsed_html:
            df = parser.extract_financial_info(parsed_html, current_year)
            
            print(f"==================== {current_year}년 관계기업 재무정보 파싱 결과 ====================")
            if df is not None:
                display(df) 
                
                # 추출된 DataFrame을 리스트에 추가
                all_dfs.append(df)
            else:
                print("테이블을 찾을 수 없습니다.")
    else:
        print(f"\n파일을 찾을 수 없습니다: {file_path}")

# 루프가 끝난 후 모든 DataFrame을 하나로 합치기
if all_dfs:
    # pd.concat()을 사용해 리스트의 모든 DataFrame을 병합
    final_df = pd.concat(all_dfs, ignore_index=False) # index=True 옵션과 호환되도록 ignore_index=False로 설정
    
    # 최종 DataFrame을 CSV 파일로 저장
    final_csv_filename = "outputs/전체_관계기업_재무정보.csv"
    final_df.to_csv(final_csv_filename, index=True, encoding='utf-8-sig')
    
    print("\n==================== 전체 데이터 합치기 완료 ====================")
    display(final_df)
    print(f"'{final_csv_filename}' 파일로 저장되었습니다.")
else:
    print("\n추출된 데이터가 없어 파일을 저장할 수 없습니다.")

In [ ]:
import os
import re
import pandas as pd
from IPython.display import display

# parser 인스턴스는 이미 생성되었다고 가정합니다.
# parser = AuditReportParser()

# 출력 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# 모든 연도의 DataFrame을 저장할 빈 리스트
all_dfs = []

for year in range(2014, 2025):
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
        year_from_filename = int(re.search(r'감사보고서_(\d{4})\.htm', file_path).group(1))
        
        if parsed_html:
            print(f"==================== {year}년 관계기업 투자주식의 내역 파싱 결과 ====================")
            df = parser.extract_specific_investment_table(parsed_html, year_from_filename)
            
            if df is not None:
                display(df)
                # 추출된 DataFrame을 리스트에 추가
                all_dfs.append(df)
            else:
                print("테이블을 찾을 수 없습니다.")
    else:
        print(f"\n파일을 찾을 수 없습니다: {file_path}")

# 루프가 끝난 후 모든 DataFrame을 하나로 합치기
if all_dfs:
    # pd.concat()을 사용해 리스트의 모든 DataFrame을 병합합니다.
    final_df = pd.concat(all_dfs, ignore_index=False)
    
    # 최종 DataFrame을 CSV 파일로 저장
    final_csv_filename = "outputs/전체_관계기업_투자주식_내역.csv"
    final_df.to_csv(final_csv_filename, index=True, encoding='utf-8-sig')
    
    print("\n==================== 전체 데이터 합치기 완료 ====================")
    display(final_df)
    print(f"'{final_csv_filename}' 파일로 저장되었습니다.")
else:
    print("\n추출된 데이터가 없어 파일을 저장할 수 없습니다.")

In [ ]:
import os
import re
import pandas as pd
from IPython.display import display

# parser 인스턴스는 이미 생성되었다고 가정합니다.
# parser = AuditReportParser()

# 출력 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# 모든 연도의 DataFrame을 저장할 빈 리스트
all_dfs = []

for year in range(2014, 2025):
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
        year_from_filename = int(re.search(r'감사보고서_(\d{4})\.htm', file_path).group(1))
        
        if parsed_html:
            print(f"==================== {year}년 유형자산 변동내역 파싱 결과 ====================")
            df = parser.extract_tangible_assets_table(parsed_html, year_from_filename)
            
            if df is not None:
                display(df)
                # 추출된 DataFrame을 리스트에 추가
                all_dfs.append(df)
            else:
                print("테이블을 찾을 수 없습니다.")
    else:
        print(f"\n파일을 찾을 수 없습니다: {file_path}")

# 루프가 끝난 후 모든 DataFrame을 하나로 합치기
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=False)
    
    final_csv_filename = "outputs/전체_유형자산_변동내역.csv"
    final_df.to_csv(final_csv_filename, index=True, encoding='utf-8-sig')
    
    print("\n==================== 전체 데이터 합치기 완료 ====================")
    display(final_df)
    print(f"'{final_csv_filename}' 파일로 저장되었습니다.")
else:
    print("\n추출된 데이터가 없어 파일을 저장할 수 없습니다.")

In [40]:
parser = AuditReportParser()

# 출력 파일을 저장할 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

for year in range(2014, 2025):
    # 파일 경로 설정 (경로를 사용자의 환경에 맞게 수정해주세요)
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    # 파일 존재 여부 확인 후 파싱 및 출력
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
    year_from_filename = int(re.search(r'감사보고서_(\d{4})\.htm', file_path).group(1))
    if parsed_html:
        print(f"==================== {year}년 무형자산 변동내역 파싱 결과 ====================")
        df = parser.extract_intangible_assets_table(parsed_html,year_from_filename)
            
        if df is not None:
            display(df)
            csv_filename = f"outputs/3-b {year}.csv"
            df.to_csv(csv_filename, index=True, encoding='utf-8-sig')
            print(f"'{csv_filename}' 파일로 저장되었습니다.")
        else:
            print("테이블을 찾을 수 없습니다.")
else:
    print(f"파일을 찾을 수 없습니다: {file_path}")

==================== 2014년 무형자산 변동내역 파싱 결과 ====================


,연도,구분,산업재산권,개발비,회원권,영업권,기타의무형자산,계,단위
0,2014,기초장부가액,"827,953","752,669","145,332","79,277","690,071","2,495,302",백만원
1,2014,내부개발에의한취득,,"940,001",,,,"940,001",백만원
2,2014,개별취득,"271,831",,"4,380",,"37,896","314,107",백만원
3,2014,상각,"136,339","396,078",,,"359,111","891,528",백만원
4,2014,처분/폐기,"39,126",,,,370,"39,496",백만원
5,2014,손상,,"56,659",,,,"56,659",백만원
6,2014,기타,"1,415",,,,"288,422","289,837",백만원
7,2014,기말장부가액,"925,734","1,239,933","149,712","79,277","656,908","3,051,564",백만원


'outputs/3-b 2014.csv' 파일로 저장되었습니다.
==================== 2015년 무형자산 변동내역 파싱 결과 ====================


,연도,구분,산업재산권,개발비,회원권,영업권,기타의무형자산,계,단위
0,2015,기초장부가액,"925,734","1,239,933","149,712","79,277","656,908","3,051,564",백만원
1,2015,내부개발에의한취득,,"1,143,059",,,,"1,143,059",백만원
2,2015,개별취득,"198,655",,"1,929",,"22,031","222,615",백만원
3,2015,상각,"151,607","607,526",,,"241,405","1,000,538",백만원
4,2015,처분/폐기,"22,036",,48,,364,"22,448",백만원
5,2015,손상,,"76,703",,"79,277",,"155,980",백만원
6,2015,기타,247,"1,218",,,"170,422","168,957",백만원
7,2015,기말장부가액,"950,499","1,697,545","151,593",,"607,592","3,407,229",백만원


'outputs/3-b 2015.csv' 파일로 저장되었습니다.
==================== 2016년 무형자산 변동내역 파싱 결과 ====================


...

'outputs/3-b 2022.csv' 파일로 저장되었습니다.
==================== 2023년 무형자산 변동내역 파싱 결과 ====================


,연도,구분,산업재산권,개발비,회원권,기타,계,단위
0,2023,기초장부금액,"1,193,286","85,018","199,870","7,083,250","8,561,424",백만원
1,2023,개별취득,"371,570",,"1,069","4,174,044","4,546,683",백만원
2,2023,상각,"229,849","85,018",,"2,299,322","2,614,189",백만원
3,2023,처분ㆍ폐기,"40,944",,"7,850",560,"49,354",백만원
4,2023,손상환입,"6,265",,"3,738","3,114","5,641",백만원
5,2023,기타,,,,"1,288","1,288",백만원
6,2023,기말장부금액,"1,287,798",,"196,827","8,955,586","10,440,211",백만원


'outputs/3-b 2023.csv' 파일로 저장되었습니다.
==================== 2024년 무형자산 변동내역 파싱 결과 ====================


,연도,구분,산업재산권,개발비,회원권,기타,계,단위
0,2024,기초장부금액,"1,287,798",,"196,827","8,955,586","10,440,211",백만원
1,2024,개별취득,"270,328",,"13,531","2,213,977","2,497,836",백만원
2,2024,상각,"236,511",,,"2,195,761","2,432,272",백만원
3,2024,처분ㆍ폐기,"33,661",,,"16,040","49,701",백만원
4,2024,손상환입,,,826,,826,백만원
5,2024,기타,741,,,"42,449","41,708",백만원
6,2024,기말장부금액,"1,287,213",,"209,532","9,000,211","10,496,956",백만원


'outputs/3-b 2024.csv' 파일로 저장되었습니다.
파일을 찾을 수 없습니다: 삼성전자_감사보고서_2014_2024/감사보고서_2024.htm


In [41]:
import os
import re
import pandas as pd
from IPython.display import display

# parser 인스턴스는 이미 생성되었다고 가정합니다.
# parser = AuditReportParser()

# 출력 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# 모든 연도의 DataFrame을 저장할 빈 리스트
all_dfs = []

for year in range(2014, 2025):
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
        year_from_filename = int(re.search(r'감사보고서_(\d{4})\.htm', file_path).group(1))
        
        if parsed_html:
            print(f"==================== {year}년 무형자산 변동내역 파싱 결과 ====================")
            df = parser.extract_intangible_assets_table(parsed_html, year_from_filename)
            
            if df is not None:
                display(df)
                # 추출된 DataFrame을 리스트에 추가
                all_dfs.append(df)
            else:
                print("테이블을 찾을 수 없습니다.")
    else:
        print(f"\n파일을 찾을 수 없습니다: {file_path}")

# 루프가 끝난 후 모든 DataFrame을 하나로 합치기
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=False)
    
    final_csv_filename = "outputs/전체_무형자산_변동내역.csv"
    final_df.to_csv(final_csv_filename, index=True, encoding='utf-8-sig')
    
    print("\n==================== 전체 데이터 합치기 완료 ====================")
    display(final_df)
    print(f"'{final_csv_filename}' 파일로 저장되었습니다.")
else:
    print("\n추출된 데이터가 없어 파일을 저장할 수 없습니다.")

==================== 2014년 무형자산 변동내역 파싱 결과 ====================


,연도,구분,산업재산권,개발비,회원권,영업권,기타의무형자산,계,단위
0,2014,기초장부가액,"827,953","752,669","145,332","79,277","690,071","2,495,302",백만원
1,2014,내부개발에의한취득,,"940,001",,,,"940,001",백만원
2,2014,개별취득,"271,831",,"4,380",,"37,896","314,107",백만원
3,2014,상각,"136,339","396,078",,,"359,111","891,528",백만원
4,2014,처분/폐기,"39,126",,,,370,"39,496",백만원
5,2014,손상,,"56,659",,,,"56,659",백만원
6,2014,기타,"1,415",,,,"288,422","289,837",백만원
7,2014,기말장부가액,"925,734","1,239,933","149,712","79,277","656,908","3,051,564",백만원


==================== 2015년 무형자산 변동내역 파싱 결과 ====================


,연도,구분,산업재산권,개발비,회원권,영업권,기타의무형자산,계,단위
0,2015,기초장부가액,"925,734","1,239,933","149,712","79,277","656,908","3,051,564",백만원
1,2015,내부개발에의한취득,,"1,143,059",,,,"1,143,059",백만원
2,2015,개별취득,"198,655",,"1,929",,"22,031","222,615",백만원
3,2015,상각,"151,607","607,526",,,"241,405","1,000,538",백만원
4,2015,처분/폐기,"22,036",,48,,364,"22,448",백만원
5,2015,손상,,"76,703",,"79,277",,"155,980",백만원
6,2015,기타,247,"1,218",,,"170,422","168,957",백만원
7,2015,기말장부가액,"950,499","1,697,545","151,593",,"607,592","3,407,229",백만원


==================== 2016년 무형자산 변동내역 파싱 결과 ====================


...

==================== 2024년 무형자산 변동내역 파싱 결과 ====================


,연도,구분,산업재산권,개발비,회원권,기타,계,단위
0,2024,기초장부금액,"1,287,798",,"196,827","8,955,586","10,440,211",백만원
1,2024,개별취득,"270,328",,"13,531","2,213,977","2,497,836",백만원
2,2024,상각,"236,511",,,"2,195,761","2,432,272",백만원
3,2024,처분ㆍ폐기,"33,661",,,"16,040","49,701",백만원
4,2024,손상환입,,,826,,826,백만원
5,2024,기타,741,,,"42,449","41,708",백만원
6,2024,기말장부금액,"1,287,213",,"209,532","9,000,211","10,496,956",백만원



==================== 전체 데이터 합치기 완료 ====================


,연도,구분,산업재산권,개발비,회원권,영업권,기타의무형자산,계,단위,기타
0,2014,기초장부가액,"827,953","752,669","145,332","79,277","690,071","2,495,302",백만원,NaN
1,2014,내부개발에의한취득,,"940,001",,,,"940,001",백만원,NaN
2,2014,개별취득,"271,831",,"4,380",,"37,896","314,107",백만원,NaN
3,2014,상각,"136,339","396,078",,,"359,111","891,528",백만원,NaN
4,2014,처분/폐기,"39,126",,,,370,"39,496",백만원,NaN
...,...,...,...,...,...,...,...,...,...,...
2,2024,상각,"236,511",,,NaN,NaN,"2,432,272",백만원,"2,195,761"
3,2024,처분ㆍ폐기,"33,661",,,NaN,NaN,"49,701",백만원,"16,040"
4,2024,손상환입,,,826,NaN,NaN,826,백만원,
5,2024,기타,741,,,NaN,NaN,"41,708",백만원,"42,449"


'outputs/전체_무형자산_변동내역.csv' 파일로 저장되었습니다.
